In [1]:
import tensorflow as tf
import numpy as np

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('mnist/', one_hot=True)

Extracting mnist/train-images-idx3-ubyte.gz
Extracting mnist/train-labels-idx1-ubyte.gz
Extracting mnist/t10k-images-idx3-ubyte.gz
Extracting mnist/t10k-labels-idx1-ubyte.gz


In [3]:
learning_rate = 0.001
batch_size = 128

n_input = 28
n_steps = 28
n_hidden = 128
n_classes = 10

In [4]:
x = tf.placeholder(tf.float32, [None, n_steps,n_input])
y = tf.placeholder(tf.float32, [None, n_classes])

In [5]:
output, _ = tf.nn.dynamic_rnn(
            tf.contrib.rnn.GRUCell(n_hidden),
            x,
            dtype=tf.float32,
            sequence_length = batch_size*[n_input],
        )

In [6]:
output.get_shape()

TensorShape([Dimension(None), Dimension(28), Dimension(128)])

In [7]:
index = tf.range(0,batch_size)*n_steps + (n_input - 1)
flat = tf.reshape(output,[-1,int(output.get_shape()[2])])
last = tf.gather(flat,index)

In [8]:
num_classes = int(y.get_shape()[1])
weight = tf.Variable(tf.truncated_normal([n_hidden, num_classes], stddev=0.01))
bias = tf.Variable(tf.constant(0.1, shape=[num_classes]))
prediction = tf.nn.softmax(tf.matmul(last, weight) + bias)

In [9]:
cross_entropy = -tf.reduce_sum(y * tf.log(prediction))

In [10]:
optimizer = tf.train.AdamOptimizer(learning_rate, beta1=0.5)
grads = optimizer.compute_gradients(cross_entropy)
for i, (g, v) in enumerate(grads):
    if g is not None:
        grads[i] = (tf.clip_by_norm(g, 5), v)  # clip gradients
train_op = optimizer.apply_gradients(grads)

C:\Users\qiqi\Anaconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:95: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [11]:
correct_pred = tf.equal(tf.argmax(prediction,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [12]:
sess = tf.Session()

In [13]:
init = tf.global_variables_initializer() 
sess.run(init) 
for step in range(1300): 
    batch_x, batch_y = mnist.train.next_batch(batch_size) 
    batch_x = batch_x.reshape((batch_size,n_steps,n_input))
    sess.run(train_op, feed_dict={x: batch_x, y: batch_y})
    if step % 50 == 0:
        acc = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y})
        loss = sess.run(cross_entropy, feed_dict={x: batch_x, y: batch_y})
        print("Iter " + str(step) + ", Minibatch Loss= " +  "{:.6f}".format(loss) + ", Training Accuracy= " +  "{:.5f}".format(acc))
print("Optimization Finished!")

Iter 0, Minibatch Loss= 293.964844, Training Accuracy= 0.25000
Iter 50, Minibatch Loss= 166.142029, Training Accuracy= 0.60156
Iter 100, Minibatch Loss= 100.657265, Training Accuracy= 0.71875
Iter 150, Minibatch Loss= 77.566505, Training Accuracy= 0.82031
Iter 200, Minibatch Loss= 80.115059, Training Accuracy= 0.78906
Iter 250, Minibatch Loss= 42.491905, Training Accuracy= 0.90625
Iter 300, Minibatch Loss= 50.145142, Training Accuracy= 0.87500
Iter 350, Minibatch Loss= 35.765850, Training Accuracy= 0.89062
Iter 400, Minibatch Loss= 26.150940, Training Accuracy= 0.94531
Iter 450, Minibatch Loss= 23.897655, Training Accuracy= 0.93750
Iter 500, Minibatch Loss= 19.633280, Training Accuracy= 0.95312
Iter 550, Minibatch Loss= 17.638645, Training Accuracy= 0.96875
Iter 600, Minibatch Loss= 14.586130, Training Accuracy= 0.98438
Iter 650, Minibatch Loss= 12.010118, Training Accuracy= 0.98438
Iter 700, Minibatch Loss= 15.924150, Training Accuracy= 0.96094
Iter 750, Minibatch Loss= 21.219090, Tra

In [14]:
test_x = mnist.test.images
test_x = test_x.reshape((-1,n_steps,n_input))
test_y = mnist.test.labels

In [15]:
acc = sess.run(accuracy, feed_dict={x: test_x[:128], y: test_y[:128]})
print(acc)

0.992188
